In [12]:
# imports
import os
import pandas as pd

In [47]:
# reading in the data
crime_data_dict = {}
rows_to_skip_dict = {'2010': 7,
                    '2011': 7,
                    '2012': 7,
                    '2013': 10,
                    '2014': 9,
                    '2015': 10,
                    '2016': 9,
                    '2017': 10,
                    '2018': 10,
                    '2019': 8}
for year in range(2010, 2020):
    year = str(year)
    FILE_PATH = os.path.join(os.getcwd(), '..', 'raw_data', f'city_crime_data_{year}.csv')
    crime_data_dict[year] = pd.read_csv(FILE_PATH, skiprows=3, skipfooter=rows_to_skip_dict[year])
    crime_data_dict[year].columns = ['state_name', 'city_name', 'violent_crime', 'murder_and_nonnegligent_homicide',
                'rape', 'robbery', 'aggravated_assault', 'property_crime', 'burglary',
                'larceny_theft', 'motor_vehicle_theft', 'arson']

In [78]:
# data cleaning
df = crime_data_dict['2018']
df['state_name'] = df['state_name'].fillna(method='ffill')
df['state_name'] = df['state_name'].apply(lambda x: ''.join([letter for letter in x if not letter.isdigit()]))
df['state_name'] = df['state_name'].apply(lambda x: x.capitalize())
df['city_name'] = df['city_name'].apply(lambda x: ''.join([letter for letter in str(x) if not letter.isdigit()]))
df = df.fillna('n/a')
for column in df.columns[2:]:
    df[f'{column}'] = df[f'{column}'].apply(lambda x: str(x))
    df[f'{column}'] = df[f'{column}'].apply(lambda x: float(x.replace(',', '')) if x != 'n/a' else x)
df

,state_name,city_name,violent_crime,murder_and_nonnegligent_homicide,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson
0,Alabama,Abbeville,18,0,2,0,16,49,14,33,2,n/a
1,Alabama,Adamsville,19,0,1,4,14,289,42,230,17,n/a
2,Alabama,Alabaster,92,0,2,10,80,579,56,497,26,n/a
3,Alabama,Albertville,24,0,6,10,8,802,194,492,116,n/a
4,Alabama,Alexander City,314,2,5,15,292,610,92,484,34,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
9247,Wyoming,Sheridan,17,1,2,0,14,313,39,261,13,1
9248,Wyoming,Thermopolis,13,0,0,0,13,64,10,48,6,0
9249,Wyoming,Torrington,35,0,0,0,35,73,12,57,4,0
9250,Wyoming,Wheatland,3,0,0,0,3,56,19,33,4,0


In [71]:
crime_data_dict['2018']

,state_name,city_name,violent_crime,murder_and_nonnegligent_homicide,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,arson
0,Alabama,Abbeville,18,0.0,2,0,16,49,14,33,2,NaN
1,Alabama,Adamsville,19,0.0,1,4,14,289,42,230,17,NaN
2,Alabama,Alabaster,92,0.0,2,10,80,579,56,497,26,NaN
3,Alabama,Albertville,24,0.0,6,10,8,802,194,492,116,NaN
4,Alabama,Alexander City,314,2.0,5,15,292,610,92,484,34,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9247,Wyoming,Sheridan,17,1.0,2,0,14,313,39,261,13,1
9248,Wyoming,Thermopolis,13,0.0,0,0,13,64,10,48,6,0
9249,Wyoming,Torrington,35,0.0,0,0,35,73,12,57,4,0
9250,Wyoming,Wheatland,3,0.0,0,0,3,56,19,33,4,0
